## 数据处理

In [2]:
import pandas as pd
import io
import re
from datetime import datetime, timedelta
from typing import List, Union, Optional
from datetime import datetime

import os, re, json, time
import typing as T
import requests

from pathlib import Path
from openpyxl import Workbook, load_workbook

from openpyxl.utils import get_column_letter
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from data_processing import load_and_process,build_jsonl_for_range, save_jsonl


In [3]:
## 研发字典
speaker_map = {
    "16186514":   "peter本尊",
    "1655611808": "运营绾绾",
    "2073820674": "沙利文老师",
    "2726067525": "milissa",
}
## 客服字典
MAPPING_FILE = "mapping地球1.xlsx"

##QQ的txt文件
pathtxt   = "1219《欢迎来到地球》测试2群.txt"

# 设定时间范围
start_time = "2025-12-17 00:00:00"
end_time   = "2025-12-18 00:00:00"


# 1) 拿到 JSONL（列表）
jsonl_lines01 = build_jsonl_for_range(
    pathtxt=pathtxt,
    mapping_file=MAPPING_FILE,
    speaker_map=speaker_map,
    start_time=start_time,
    end_time=end_time,
    return_str=False,   # 返回 list[str]
)

print(len(jsonl_lines01), "lines")
print(jsonl_lines01[0])



5498 lines
{"发言日期": "2025-12-17", "发言时间": "00:00:33", "玩家ID": "/mg幻风(43507639)", "玩家消息": "[图片]"}


## 大模型分类

## 定义

In [4]:
"""
批处理 10000 条聊天数据（每批 100 条）：
- 模型#1：过滤非游戏相关（只保留相关 JSON 行，原样输出）
- 模型#2：提取高讨论的发言并分析
- 结果按word格式文档输出
"""
from model_classifyV1_Copy1_Copy1 import (
    load_system_prompt,
    build_user_prompt_filter,build_user_prompt_clsuter,call_ark_chat_completions,
    extract_valid_json_lines,add_index_to_jsonl_lines,count_output_filter_stats,get_covered_indices_from_cluster_output,
    aggregate_cluster_outputs,build_user_prompt_cluster_agg,assign_global_cluster_ids,
    extract_top5_heat_clusters,attach_discussion_points,extract_cluster_stats,append_daily_top5_to_version_jsonl,infer_date_for_batch,
    build_user_prompt_subcluster_opinion,
    print_mech_time_from_top5,get_dialogs_lines_by_fayan_time_debug,merge_top5_with_opinions_numbered,parse_opinion_output_to_list,
    ensure_time_axis_key,ensure_subcluster_list_key,match_dialogs_by_time, parse_jsonl_text
)

## 设置参数

In [5]:
# ============= 你的模型与文件配置（改这里） =============
API_URL   = "https://ark.cn-beijing.volces.com/api/v3/chat/completions" 
API_KEY = "de91deb0-aae6-46cb-bac0-17ac3b6107f5" #API
V3_MODEL_ID= "ep-20251020160142-5d7hp"#接入点
V3_1_MODEL_ID = "ep-20251020160025-9p5tj"#接入点
R1_MODEL_ID = "ep-20251020160103-5n6g2"#接入点

PROMPT_MD_PATH01 = Path("提示词1.md") # 模型#1 system 提示词（筛相关）
PROMPT_MD_PATH02 = Path("2话题分类.md") # 模型#2 system 提示词（分话题）
PROMPT_MD_PATH03 = Path("3日聚合.md") # 模型#3 system 提示词（日聚合）
PROMPT_MD_PATH04 = Path("2话题分类和总结.md") #模型#4 system 提示词（观点分析）
VERSION_TOP5_JSONL = "version_daily_top5_with_opinion.jsonl"
BATCH_SIZE       = 300
SLEEP_BETWEEN    = 1   # 每批之间的间隔，防止QPS触发限流；按需调整
RETRIES          = 2
TEMPERATURE      = 0.20
MAX_TOKENS       = 16384
TIMEOUT_SEC      = 600
# =====================================================

## 1.检验每日话提簇分类输出

In [15]:
import json
import time
from tqdm import tqdm



# ===========================================================
# =============== 主循环：模型#1 + 模型#2 ====================
# ===========================================================

# 读取系统提示
system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做话题簇
# 1⃣ 给原始 jsonl 每条加 _idx
jsonl_lines01_indexed = add_index_to_jsonl_lines(jsonl_lines01)

total = len(jsonl_lines01_indexed)

PRINT_UNCLUSTERED = False
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

# 统计总量用的（可选）
total_filtered_lines = 0         # 模型#1 输出总行数之和（含客服）
total_filtered_player_lines = 0  # 模型#1 输出玩家行数之和
total_covered_idx = 0            # 被话题簇覆盖的原始发言总数（按 _idx 去重）


for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    # 本批原始发言（已带 _idx）
    batch_lines = jsonl_lines01_indexed[start:end]

    # 本批原始 _idx 集合，用来和话题簇覆盖做差集
    batch_original_idx = set()
    for line in batch_lines:
        try:
            obj = json.loads(line)
            batch_original_idx.add(int(obj["_idx"]))
        except Exception:
            continue

    try:
        # ========== 模型 #1：筛相关 ==========
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue

        # 统计筛相关之后的行数：总行数 & 玩家行数
        filter_total_lines, filter_player_lines = count_output_filter_stats(output_filter)
        total_filtered_lines += filter_total_lines
        total_filtered_player_lines += filter_player_lines

        tqdm.write(
            f"[批次 {b+1}] 模型#1 输出总行数：{filter_total_lines}，"
            f"其中玩家发言：{filter_player_lines} 条（不去重）"
        )

        # ========== 模型 #2：话题簇 ==========
        user_prompt2 = build_user_prompt_clsuter(output_filter)
        output_cluster = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_cluster:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue

        # 打印本批的话题簇输出（原样）
        print(f"\n===== 批次 {b+1}/{total_batches} 话题簇输出 =====")
        print(output_cluster)
        print("===========================================\n")

        # 计算：本批有哪些 _idx 被话题簇覆盖（从“发言行号列表”解析）
        covered_idx = get_covered_indices_from_cluster_output(output_cluster)
        total_covered_idx += len(covered_idx)

        # 和“本批原始发言 _idx 集合”做差集，看有哪些没被覆盖
        unclustered_idx = batch_original_idx - covered_idx

        tqdm.write(
            f"[批次 {b+1}] 话题簇覆盖原始发言条数：{len(covered_idx)}，"
            f"未被覆盖条数：{len(unclustered_idx)}"
        )

        # ⭐⭐ 新增：打印“未被话题簇覆盖”的发言信息（最多前 20 条）
        if PRINT_UNCLUSTERED and unclustered_idx:
            # 先把这一批中未覆盖的发言对象捞出来
            unclustered_records = []
            for line in batch_lines:
                try:
                    obj = json.loads(line)
                except Exception:
                    continue
                idx_val = obj.get("_idx")
                try:
                    idx_int = int(idx_val)
                except Exception:
                    continue
                if idx_int in unclustered_idx:
                    unclustered_records.append(obj)

            # 按 _idx 排序，避免乱序
            unclustered_records.sort(key=lambda x: int(x.get("_idx", 0)))

            print(f"\n[批次 {b+1}] 未被话题簇覆盖的原始发言（最多前20条）：")
            for rec in unclustered_records[:20]:
                idx = rec.get("_idx")
                date = rec.get("发言日期") or rec.get("日期") or ""
                t = rec.get("发言时间") or rec.get("时间") or ""
                speaker = rec.get("发言人ID") or rec.get("玩家ID") or rec.get("角色ID") or ""
                msg = (
                    rec.get("玩家消息")
                    or rec.get("发言内容")
                    or rec.get("玩家发言")
                    or rec.get("消息")
                    or ""
                )
                print(f"- _idx={idx} [{date} {t}] {speaker}: {msg}")
            print()  # 换行分隔一下

    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    # 防止 QPS 过高
    time.sleep(SLEEP_BETWEEN)

print("\n✅ 全部批次处理完成！")
print(f"原始输入总数：{total}")
print(f"模型#1 筛相关后总行数：{total_filtered_lines}，其中玩家发言：{total_filtered_player_lines} 条")
print(f"模型#2 话题簇累计覆盖原始发言条数：{total_covered_idx}")

准备处理 141 条，共 1 批（每批 300 条）。


🔥 批处理进度:   0%|                                                                                                                                     | 0/1 [02:39<?, ?批/s]      

[批次 1] 模型#1 输出总行数：89，其中玩家发言：89 条（不去重）


🔥 批处理进度:   0%|                                                                                                                                     | 0/1 [02:46<?, ?批/s]      


===== 批次 1/1 话题簇输出 =====
{"话题簇":"停服维护补偿讨论（2025-11-19 15:02:03-15:11:27）","核心对象/机制":"停服维护补偿机制，玩家讨论维护时长与补偿期望（100抽、50抽、石币、代币等）"}
{"话题簇":"游戏下载与安装问题（2025-11-19 15:02:22-15:13:20）","核心对象/机制":"游戏安装包下载与更新机制，玩家询问下载链接、是否需要重新下载等问题"}
{"话题簇":"角色技能更新讨论（2025-11-19 15:02:58-15:04:02）","核心对象/机制":"猛犸象和仙人掌角色的PTSD与强迫症被动技能更新实装情况"}
{"话题簇":"客服互动与调侃（2025-11-19 15:07:27-15:14:50）","核心对象/机制":"客服服务体验，玩家与客服薏米、蓝桉的互动调侃及无关游戏内容的闲聊"}

[批次 1] 话题簇覆盖原始发言条数：0，未被覆盖条数：141


🔥 批处理进度: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:47<00:00, 167.86s/批]


✅ 全部批次处理完成！
原始输入总数：141
模型#1 筛相关后总行数：89，其中玩家发言：89 条
模型#2 话题簇累计覆盖原始发言条数：0


## 加讨论观点分析的版本测试

In [6]:
from tqdm import tqdm
import time
import json
from pathlib import Path

# ... 前置：系统提示、create_intent_excel_styled(EXCEL_FILE)、jsonl_lines01 等

system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做话提簇
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)  # 话题簇聚合 / 校正（智能体4）
system_prompt04 = load_system_prompt(PROMPT_MD_PATH04)  # 话题簇聚合 / 校正（智能体4）
# 用于存放当天所有批次的话题簇 JSON 行（给智能体4用）
batch_cluster_outputs  = []
all_opinions = []
total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

written_total = 0

for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
        # --- 模型 #1：筛相关 ---
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue
        filter_count = sum(1 for line in output_filter.splitlines() if line.strip())
        tqdm.write(f"[批次 {b+1}] 模型#1 筛后保留 {filter_count} 条")
        written_total += filter_count   # 如果不需要总数，可以删掉这一行
        
        # --- 模型 #2：话题簇划分 ---
        user_prompt2 = build_user_prompt_clsuter(output_filter)
        output_cluster = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_cluster:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue
        try:
            cluster_json_list = []
            for line in output_cluster.strip().splitlines():
                line = line.strip()
                if not line.startswith("{"):
                    continue
                obj = json.loads(line)
                # 将 "话题簇1" → "话题簇"，便于后续统一处理
                for key in list(obj.keys()):
                    if key.startswith("话题簇") and key != "话题簇":
                        obj["话题簇"] = obj.pop(key)
                cluster_json_list.append(obj)

            # 假设从输出中拿日期
            date_str = infer_date_for_batch(cluster_json_list, batch_lines)  # 你可以从上下文获得更准确值
            batch_id = f"B{b+1}"
            cluster_json_list = assign_global_cluster_ids(cluster_json_list, date_str, batch_id)
        
            output_cluster_with_ids = "\n".join(json.dumps(c, ensure_ascii=False) for c in cluster_json_list)
            batch_cluster_outputs.append(output_cluster_with_ids)
        except Exception as e:
            tqdm.write(f"[批次 {b+1}] ⚠ 添加 _cluster_id 失败：{e}")
            continue

       
       
    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    time.sleep(SLEEP_BETWEEN)



# --- 每日话提簇聚合成jsonl ---
all_cluster = aggregate_cluster_outputs(batch_cluster_outputs)

# --- 模型 #3：日话提簇聚合 ---
user_prompt3 = build_user_prompt_cluster_agg(all_cluster)
output_cluster_agg = call_ark_chat_completions(
    api_url=API_URL,
    api_key=API_KEY,
    model=V3_1_MODEL_ID,           # 或你给智能体4选的模型
    system_prompt=system_prompt03,  # 话题簇聚合/校正提示词
    user_prompt=user_prompt3,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    timeout=TIMEOUT_SEC,
    retries=RETRIES,
)

# --- 每日话提簇聚合成jsonl ---
all_cluster = aggregate_cluster_outputs(batch_cluster_outputs)

# --- 模型 #3：日话提簇聚合 ---
user_prompt3 = build_user_prompt_cluster_agg(all_cluster)
output_cluster_agg = call_ark_chat_completions(
    api_url=API_URL,
    api_key=API_KEY,
    model=V3_1_MODEL_ID,
    system_prompt=system_prompt03,
    user_prompt=user_prompt3,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    timeout=TIMEOUT_SEC,
    retries=RETRIES,
)



# --- 筛选每日发言热度top5 ---
# 子话题簇（来自 all_cluster）
# ✅ 2) 再解析聚合簇（来自 output_cluster_agg）
# 聚合簇（模型#3 输出）
parsed_clusters = parse_jsonl_text(output_cluster_agg)

# 子话题簇（all_cluster 已经是 jsonl，但也用统一解析更安全）
parsed_subclusters = parse_jsonl_text(all_cluster)
# ✅ 3) 最后兜底补齐“时间轴”（避免 None.split）
fixed_cnt = 0
fixed_sub_cnt = 0
for c in parsed_clusters:
    if ensure_time_axis_key(c, verbose=False):
        fixed_cnt += 1
    if ensure_subcluster_list_key(c):
        fixed_sub_cnt += 1
print(f"✅ 已自动补齐/修复 时间轴 的聚合簇数量：{fixed_cnt}")
print(f"✅ 已自动补齐/修复 子话题簇列表 的聚合簇数量：{fixed_sub_cnt}")

# 计算 top5
top5_results = extract_top5_heat_clusters(parsed_clusters, jsonl_lines01, top_k=5)
final_result = attach_discussion_points(top5_results, parsed_subclusters)

# --- 根据每日讨论热度5回溯至原文分析观点共识 ---
rows = print_mech_time_from_top5(final_result, all_cluster)

# ✅ 记得初始化
all_opinions = []

for idx, r in enumerate(rows, start=1):
    mech = r.get("核心对象/机制") or ""
    full_time = (r.get("发言时间") or "").strip()

    if not mech or not full_time:
        print(f"⚠ 第 {idx} 条缺少 核心对象/机制 或 发言时间，原始行：{r}")
        continue

    # full_time 格式类似："2025-12-02 14:01:57-14:42:21"
    if " " not in full_time:
        print(f"⚠ 第 {idx} 条发言时间格式异常，无法拆分日期和时间轴：{full_time}")
        continue

    fayan_date, fayan_time = full_time.split(" ", 1)  # "2025-12-02", "14:01:57-14:42:21"

    # 用 debug 版回溯原文
    dialogs_lines = get_dialogs_lines_by_fayan_time_debug(
        jsonl_lines01,
        fayan_date,
        fayan_time,
        debug=True,
    )

    if not dialogs_lines:
        print(f"⚠ 该时间段内没有原文，跳过：{mech} | {fayan_date} {fayan_time}")
        continue

    # --- 模型 #4：分析玩家共识 ---
    user_prompt4 = build_user_prompt_subcluster_opinion(
        discussion_point=mech,
        json_lines=dialogs_lines,
    )

    try:
        opinion_output = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt04,
            user_prompt=user_prompt4,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
    except Exception as e:
        print("❌ 调用模型4出错:", e)
        continue

    opinions_this_mech = parse_opinion_output_to_list(opinion_output)
    all_opinions.extend(opinions_this_mech)

merged_top5 = merge_top5_with_opinions_numbered(final_result, all_opinions)

for row in merged_top5:
    print(json.dumps(row, ensure_ascii=False, indent=2))


# 输出查看：
#for row in final_result:
    #print(json.dumps(row, ensure_ascii=False, indent=2))

#append_daily_top5_to_version_jsonl(
    #final_result,
    #version_jsonl_path=VERSION_TOP5_JSONL # 这个路径你可以按版本号动态改
#)



    
    

准备处理 5498 条，共 19 批（每批 300 条）。


🔥 批处理进度:   0%|                                                                                                                                     | 0/19 [02:45<?, ?批/s]      

[批次 1] 模型#1 筛后保留 92 条


🔥 批处理进度:   5%|██████▌                                                                                                                     | 1/19 [03:40<54:11, 180.65s/批]      

[批次 2] 模型#1 筛后保留 18 条


...

[批次 18] 模型#1 筛后保留 286 条


🔥 批处理进度:  95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋      | 18/19 [1:58:20<07:47, 467.95s/批]      

[批次 19] 模型#1 筛后保留 87 条


🔥 批处理进度: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [1:58:39<00:00, 374.73s/批]


✅ 已自动补齐/修复 时间轴 的聚合簇数量：0
✅ 已自动补齐/修复 子话题簇列表 的聚合簇数量：0
{
  "聚合话题簇": "游戏下载与注册时间咨询",
  "日期": "2025-12-17",
  "时间轴": "10:56:30-11:16:04、11:16:28-11:24:30、11:39:27-11:39:56、12:05:29-13:33:08、13:39:35-13:40:03、16:40:36-16:51:15、17:09:05-17:09:53、17:52:17-18:02:38、20:16:55-20:18:34、23:36:28-23:41:13",
  "发言玩家总数": 90,
  "发言总数": 853,
  "热度评分": 2628.55,
  "讨论点列表": [
    {
      "讨论点1": "地球游戏测试版下载链接发放时间与注册开放时间",
      "玩家观点": [
        "1：多名玩家询问下载链接和注册开放时间，表现出对游戏测试的期待和急切心情",
        "2：个别玩家对下午开放下载和注册的时间安排表示接受和等待",
        "3：少数玩家担心自己时间冲突，询问是否可以延后注册或下载"
      ],
      "代表性玩家发言示例": [
        "@青瓷游戏客服-菜头 啥时候开启注册啊",
        "下午",
        "我要上课怎么办",
        "我下午在睡觉。",
        "一点50发链接，再等一会会"
      ]
    },
    {
      "讨论点2": "游戏下载链接发布时间与获取方式",
      "玩家观点": [
        "1：多名玩家询问下载链接的具体发布时间，客服明确回复下午发布",
        "2：若干玩家关心下载链接的获取方式，客服表示会通过公告和@全体成员的方式发布",
        "3：个别玩家询问游戏平台兼容性，客服确认支持安卓设备"
      ],
      "代表性玩家发言示例": [
        "下午才能玩吗",
        "发链接会at全体吧",
        "手机能玩吗"
      ]
    },
    {
      "讨论点3":

In [8]:
print("====== DEBUG rows 中含 '降神系统运行时的性能表现与卡顿问题' 的记录 ======")
for r in rows:
    if "降神系统运行时的性能表现与卡顿问题" in (r.get("核心对象/机制") or ""):
        print(json.dumps(r, ensure_ascii=False, indent=2))


====== DEBUG rows 中含 '降神系统运行时的性能表现与卡顿问题' 的记录 ======
{
  "核心对象/机制": "降神系统运行时的性能表现与卡顿问题",
  "发言时间": "2025-12-17 16:31:26至16:32:57"
}


## 开始运行(不可动）

In [14]:
from tqdm import tqdm
import time
import json
from pathlib import Path

# ... 前置：系统提示、create_intent_excel_styled(EXCEL_FILE)、jsonl_lines01 等

system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做话提簇
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)  # 话题簇聚合 / 校正（智能体4）
system_prompt04 = load_system_prompt(PROMPT_MD_PATH04)  # 话提簇玩家观点感受分析


sub_opinion_map = {}
# 用于存放当天所有批次的话题簇 JSON 行（给智能体4用）
batch_cluster_outputs  = []
total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

written_total = 0

for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
        # --- 模型 #1：筛相关 ---
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue
        filter_count = sum(1 for line in output_filter.splitlines() if line.strip())
        tqdm.write(f"[批次 {b+1}] 模型#1 筛后保留 {filter_count} 条")
        written_total += filter_count   # 如果不需要总数，可以删掉这一行
        
        # --- 模型 #2：话题簇划分 ---
        user_prompt2 = build_user_prompt_clsuter(output_filter)
        output_cluster = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
        

        if not output_cluster:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue
        try:
            cluster_json_list = []
            for raw in output_cluster.strip().splitlines():
                line = raw.strip()
                if not line:
                    continue
                if line in ("{", "}", "},"):
                    continue
                if not line.startswith("{"):
                    continue
            try:
                obj = json.loads(line)
            except json.JSONDecodeError as e:
                print(f"[批次 {b+1}] ⚠ 解析模型#2 输出 JSON 失败：{e}，行内容：{line[:120]}")
                continue
            
            # 将 "话题簇1" → "话题簇"，便于后续统一处理
            for key in list(obj.keys()):
                if key.startswith("话题簇") and key != "话题簇":
                    obj["话题簇"] = obj.pop(key)
            cluster_json_list.append(obj)


            # 假设从输出中拿日期
            date_str = infer_date_for_batch(cluster_json_list, batch_lines)  # 你可以从上下文获得更准确值
            batch_id = f"B{b+1}"
            cluster_json_list = assign_global_cluster_ids(cluster_json_list, date_str, batch_id)
        
            output_cluster_with_ids = "\n".join(json.dumps(c, ensure_ascii=False) for c in cluster_json_list)
            batch_cluster_outputs.append(output_cluster_with_ids)
        except Exception as e:
            tqdm.write(f"[批次 {b+1}] ⚠ 添加 _cluster_id 失败：{e}")
            continue

       
       
    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    time.sleep(SLEEP_BETWEEN)



# --- 每日话提簇聚合成jsonl ---
all_cluster = aggregate_cluster_outputs(batch_cluster_outputs)

# --- 模型 #3：日话提簇聚合 ---
user_prompt3 = build_user_prompt_cluster_agg(all_cluster)
output_cluster_agg = call_ark_chat_completions(
    api_url=API_URL,
    api_key=API_KEY,
    model=V3_1_MODEL_ID,           # 或你给智能体4选的模型
    system_prompt=system_prompt03,  # 话题簇聚合/校正提示词
    user_prompt=user_prompt3,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    timeout=TIMEOUT_SEC,
    retries=RETRIES,
)


# 1) 先修 raw 文本输出（统一 key 名）
output_cluster_agg = fix_output_cluster_agg_keys(output_cluster_agg)


#每日总发言转单条jonsl格式
parsed_subclusters = [
    json.loads(line.strip()) for line in all_cluster.strip().splitlines() if line.strip()
]

# 3) 最后做兜底：确保每条都有 时间轴
fixed_cnt = 0
for c in parsed_clusters:
    if ensure_time_axis_key(c, verbose=False):
        fixed_cnt += 1
print(f"✅ 已自动补齐/修复 时间轴 的聚合簇数量：{fixed_cnt}")


# 解析原始发言 jsonl，为按时间轴筛选对话用
parsed_msgs = [json.loads(line.strip()) for line in jsonl_lines01 if line.strip()]

#每日聚合后话提簇转单条jsonl格式
parsed_clusters = [json.loads(line) for line in output_cluster_agg.strip().splitlines() if line.strip()]

#计算每日讨论热度top5
top5_results = extract_top5_heat_clusters(parsed_clusters, jsonl_lines01, top_k=5)

#######提取子话提簇讨论点###########################################################
####### 建子簇索引：_cluster_id -> 子簇对象 ########
sub_map = {}
for row in parsed_subclusters:
    cid = row.get("_cluster_id")
    if isinstance(cid, str) and cid:
        sub_map[cid] = row

####### 在 Top5 循环里，对每个【子话题簇】单独跑模型#4 ########
for cluster in top5_results:
    date = cluster["日期"]
    # 聚合簇的整体时间轴（只用来兜底）
    cluster_time_axis = cluster["时间轴"]
    cid_list = cluster.get("子话题簇列表", [])

    for cid in cid_list:
        sub = sub_map.get(cid)
        if not sub:
            print(f"\n[聚合话题簇：{cluster.get('聚合话题簇')}] 子话题簇 ID：{cid}")
            print("  ⚠ 在 sub_map / parsed_subclusters 中未找到该 _cluster_id。")
            continue

        
        # ---- 1）从子话题簇【标题】里提取时间轴 ----
        sub_title = sub.get("话题簇", "") or ""
        sub_time_axis = extract_time_axis_from_title(sub_title)

        # ---- 2）按【子话提簇时间轴】从原始发言里匹配 dialogs ----
        dialogs = match_dialogs_by_time(parsed_msgs, date, sub_time_axis)

        # ---- 3）构造 discussion_point（不打印，只用来写 prompt）----
        discussion_point = (
    
            sub.get("核心对象/机制")
            or sub.get("话题簇")
            or ""
        )
        topic_id = sub.get("_cluster_id", cid)
        
        # ---- 4）模型4 玩家发言感受分析总结----
        user_prompt4 = build_user_prompt_subcluster_opinion(
            topic_id=topic_id,
            discussion_point=discussion_point,
            dialogs=dialogs,
        )
        if not isinstance(user_prompt4, str) or not user_prompt4.strip():
            print(f"  ⚠ user_prompt4 为空或非法，跳过本子话题簇：{cid}")
            continue
        opinion_output = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,             
            system_prompt=system_prompt04,   
            user_prompt=user_prompt4,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )


       
    opinion_obj = parse_and_normalize_opinion_output(
        opinion_output=opinion_output,
        topic_id=topic_id,
        discussion_point=discussion_point,
    )
    sub_opinion_map[topic_id] = opinion_obj
# ===================================
# 组装 Top5 + 观点总结，并写入 jsonl
# ===================================



准备处理 2308 条，共 8 批（每批 300 条）。


🔥 批处理进度:   0%|                                                                                                                                                                                         | 0/8 [01:49<?, ?批/s]      

[批次 1] 模型#1 筛后保留 68 条


🔥 批处理进度:  12%|██████████████████████                                                                                                                                                          | 1/8 [10:14<13:56, 119.49s/批]      

[批次 2] 模型#1 筛后保留 264 条


...

🔥 批处理进度:  75%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                            | 6/8 [48:56<15:20, 460.17s/批]      

[批次 7] 模型#1 筛后保留 235 条


🔥 批处理进度:  88%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                      | 7/8 [54:33<07:19, 439.94s/批]      

[批次 8] 模型#1 筛后保留 205 条


🔥 批处理进度: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [55:16<00:00, 414.54s/批]


In [15]:
#每日总发言转单条jonsl格式
parsed_subclusters = [
    json.loads(line.strip()) for line in all_cluster.strip().splitlines() if line.strip()
]

# 解析原始发言 jsonl，为按时间轴筛选对话用
parsed_msgs = [json.loads(line.strip()) for line in jsonl_lines01 if line.strip()]

#每日聚合后话提簇转单条jsonl格式
parsed_clusters = [json.loads(line) for line in output_cluster_agg.strip().splitlines() if line.strip()]

#计算每日讨论热度top5
top5_results = extract_top5_heat_clusters(parsed_clusters, jsonl_lines01, top_k=5)

#######提取子话提簇讨论点###########################################################
####### 建子簇索引：_cluster_id -> 子簇对象 ########
sub_map = {}
for row in parsed_subclusters:
    cid = row.get("_cluster_id")
    if isinstance(cid, str) and cid:
        sub_map[cid] = row

####### 在 Top5 循环里，对每个【子话题簇】单独跑模型#4 ########
for cluster in top5_results:
    date = cluster["日期"]
    # 聚合簇的整体时间轴（只用来兜底）
    cluster_time_axis = cluster["时间轴"]
    cid_list = cluster.get("子话题簇列表", [])

    for cid in cid_list:
        sub = sub_map.get(cid)
        if not sub:
            print(f"\n[聚合话题簇：{cluster.get('聚合话题簇')}] 子话题簇 ID：{cid}")
            print("  ⚠ 在 sub_map / parsed_subclusters 中未找到该 _cluster_id。")
            continue

        
        # ---- 1）从子话题簇【标题】里提取时间轴 ----
        sub_title = sub.get("话题簇", "") or ""
        sub_time_axis = extract_time_axis_from_title(sub_title)

        # ---- 2）按【子话提簇时间轴】从原始发言里匹配 dialogs ----
        dialogs = match_dialogs_by_time(parsed_msgs, date, sub_time_axis)

        # ---- 3）构造 discussion_point（不打印，只用来写 prompt）----
        discussion_point = (
    
            sub.get("核心对象/机制")
            or sub.get("话题簇")
            or ""
        )
        topic_id = sub.get("_cluster_id", cid)
        
        # ---- 4）模型4 玩家发言感受分析总结----
        user_prompt4 = build_user_prompt_subcluster_opinion(
            topic_id=topic_id,
            discussion_point=discussion_point,
            dialogs=dialogs,
        )
        if not isinstance(user_prompt4, str) or not user_prompt4.strip():
            print(f"  ⚠ user_prompt4 为空或非法，跳过本子话题簇：{cid}")
            continue
        opinion_output = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,             
            system_prompt=system_prompt04,   
            user_prompt=user_prompt4,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )


       
    opinion_obj = parse_and_normalize_opinion_output(
        opinion_output=opinion_output,
        topic_id=topic_id,
        discussion_point=discussion_point,
    )
    sub_opinion_map[topic_id] = opinion_obj
# ===================================
# 组装 Top5 + 观点总结，并写入 jsonl
# ===================================



In [19]:
final_top5_with_opinion = build_daily_top5_opinion_records(
    top5_results=top5_results,
    sub_opinion_map=sub_opinion_map,
)

append_daily_top5_to_version_jsonl(
    final_result=final_top5_with_opinion,
    version_jsonl_path="version_daily_top5_with_opinion.jsonl",
)

⚠ 子话题簇 2025-12-04_B1_01 未在 sub_opinion_map 中找到模型#4结果，跳过。
⚠ 子话题簇 2025-12-04_B3_01 未在 sub_opinion_map 中找到模型#4结果，跳过。
⚠ 子话题簇 2025-12-04_B5_01 未在 sub_opinion_map 中找到模型#4结果，跳过。
✅ 已将当日 Top5（含 _idx 和 _daily_top_id）追加写入: version_daily_top5_with_opinion.jsonl


In [23]:
print(cid_list)
print(sub)

print(sub_title)
print(sub_time_axis)
print(discussion_point)
print(dialogs)
print(cid)

['2025-12-02_B1_03', '2025-12-02_B1_14', '2025-12-02_B2_07']
{'话题簇': '游戏物品异常问题反馈（2025-12-02 22:57:27-22:57:31）', '核心对象/机制': '游戏内物品异常消失问题的反馈与处理', '_cluster_id': '2025-12-02_B2_07'}
游戏物品异常问题反馈（2025-12-02 22:57:27-22:57:31）
22:57:27-22:57:31
游戏内物品异常消失问题的反馈与处理
[{'发言日期': '2025-12-02', '发言时间': '22:57:27', '玩家ID': '无为无畏(2514177080)', '玩家消息': '[图片]'}, {'发言日期': '2025-12-02', '发言时间': '22:57:31', '玩家ID': '无为无畏(2514177080)', '玩家消息': '东西没了'}]
2025-12-02_B2_07


In [13]:
try:
    obj = json.loads(line)
except json.JSONDecodeError as e:
    print(
        f"[批次 {b+1}] ⚠ 解析模型#2 输出 JSON 失败：{e}，行内容：{line!r}"
    )
    print("=== 该批次模型#2 原始完整输出（调试用） ===")
    print(output_cluster)
    print("=== ↑↑↑ 请查看这一批输出中哪里有多余的大括号/奇怪字符 ↑↑↑ ===")
    continue


SyntaxError: 'continue' not properly in loop (1099491447.py, line 10)

In [22]:
print(top5_results )

[{'聚合话题簇': '精神力恢复与驾驶员死亡机制', '子话题簇列表': ['2025-12-04_B1_01', '2025-12-04_B3_01', '2025-12-04_B5_01', '2025-12-04_B7_01'], '日期': '2025-12-04', '时间轴': '00:16:08-00:16:44、11:35:58-11:38:28、15:03:41-15:03:41、19:31:23-19:31:42', '发言玩家总数': 10, '发言总数': 36, '热度评分': 60.0}, {'聚合话题簇': '向日葵花园仓库卡关问题', '子话题簇列表': ['2025-12-04_B4_01'], '日期': '2025-12-04', '时间轴': '12:24:48-12:28:54', '发言玩家总数': 6, '发言总数': 32, '热度评分': 33.94}, {'聚合话题簇': '游戏界面BUG与显示异常', '子话题簇列表': ['2025-12-04_B8_01'], '日期': '2025-12-04', '时间轴': '23:57:14-23:59:10', '发言玩家总数': 1, '发言总数': 6, '热度评分': 2.45}, {'聚合话题簇': '地图缩放功能体验优化', '子话题簇列表': ['2025-12-04_B2_01'], '日期': '2025-12-04', '时间轴': '10:08:53-10:09:16', '发言玩家总数': 1, '发言总数': 5, '热度评分': 2.24}]


In [24]:
DEBUG_CIDS = {"2025-12-04_B1_01", "2025-12-04_B3_01", "2025-12-04_B5_01"}

for cluster in top5_results:
    date = cluster["日期"]
    cid_list = cluster.get("子话题簇列表", [])

    for cid in cid_list:
        sub = sub_map.get(cid)
        if not sub:
            print(f"[DEBUG] 子簇 {cid} 在 sub_map 中不存在")
            continue

        sub_title = sub.get("话题簇", "") or ""
        sub_time_axis = extract_time_axis_from_title(sub_title)
        dialogs = match_dialogs_by_time(parsed_msgs, date, sub_time_axis)

        discussion_point = (
            sub.get("核心对象/机制") or sub.get("话题簇") or ""
        )
        topic_id = sub.get("_cluster_id", cid)

      

        if cid in DEBUG_CIDS:
            print("\n===== DEBUG 模型#4阶段 =====")
            print("cid:", cid)
            print("sub_title:", sub_title)
            print("sub_time_axis:", sub_time_axis)
            print("dialogs 条数:", len(dialogs))
            print("user_prompt4 前 200 字:")
            print(user_prompt4[:200])
            print("opinion_output 原始内容:")
            print(opinion_output)
            print("===== END DEBUG =====\n")

        opinion_obj = parse_and_normalize_opinion_output(
            opinion_output=opinion_output,
            topic_id=topic_id,
            discussion_point=discussion_point,
        )

        if not opinion_obj:
            print(f"  ⚠ 模型#4输出中未找到合法 JSON：{cid}")
            continue

        sub_opinion_map[topic_id] = opinion_obj



===== DEBUG 模型#4阶段 =====
cid: 2025-12-04_B1_01
sub_title: 精神力恢复与驾驶员死亡机制（2025-12-04 00:16:08-00:16:44）
sub_time_axis: 00:16:08-00:16:44
dialogs 条数: 15
user_prompt4 前 200 字:
禁止使用 ```json 或 ``` 等 Markdown 代码块，禁止输出解释文字。

【输入】本次所有数据（JSONL，第一行为话题簇信息，其余为发言）：
{"话题簇ID": "2025-12-04_B1_01", "讨论点": "精神力补充方式与驾驶员死亡复活机制"}
{"发言日期": "2025-12-04", "发言时间": "00:16:08", "玩家ID": "1(34771858
opinion_output 原始内容:
{"话题簇ID": "2025-12-04_B2_01", "讨论点": "地图缩放功能的使用体验与优化建议", "玩家共识": "玩家普遍认为当前地图缩放功能体验不佳，缩放范围受限且地图过大导致操作不便", "玩家主要分歧点": "无明显分歧", "代表性玩家发言示例": ["缩放好难受", "不能再小一点么", "@有缘分. 地图大了"]}
===== END DEBUG =====


===== DEBUG 模型#4阶段 =====
cid: 2025-12-04_B3_01
sub_title: 精神值回复机制（2025-12-04 11:35:58-11:38:28）
sub_time_axis: 11:35:58-11:38:28
dialogs 条数: 17
user_prompt4 前 200 字:
禁止使用 ```json 或 ``` 等 Markdown 代码块，禁止输出解释文字。

【输入】本次所有数据（JSONL，第一行为话题簇信息，其余为发言）：
{"话题簇ID": "2025-12-04_B1_01", "讨论点": "精神力补充方式与驾驶员死亡复活机制"}
{"发言日期": "2025-12-04", "发言时间": "00:16:08", "玩家ID": "1(34771858
opinion_output 原始内容:
{"话题簇ID": "2025-12-